### Implementing linear regression in numpy.
Linear regression implemented using:
- closed form solution
- gradient descent
#### First, importing the data.

In [1]:
import pandas as pd

data = pd.read_csv('data/StudentPerformanceFactors.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6607 entries, 0 to 6606
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Hours_Studied               6607 non-null   int64 
 1   Attendance                  6607 non-null   int64 
 2   Parental_Involvement        6607 non-null   object
 3   Access_to_Resources         6607 non-null   object
 4   Extracurricular_Activities  6607 non-null   object
 5   Sleep_Hours                 6607 non-null   int64 
 6   Previous_Scores             6607 non-null   int64 
 7   Motivation_Level            6607 non-null   object
 8   Internet_Access             6607 non-null   object
 9   Tutoring_Sessions           6607 non-null   int64 
 10  Family_Income               6607 non-null   object
 11  Teacher_Quality             6529 non-null   object
 12  School_Type                 6607 non-null   object
 13  Peer_Influence              6607 non-null   obje

#### Separating features from target.

In [2]:
X = data.drop('Exam_Score', axis=1)
Y = data['Exam_Score']

#### Defining pipelines.

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.75)

num_cols = X.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X.select_dtypes(include=['object', 'category']).columns

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

from sklearn.preprocessing import OneHotEncoder

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

#### Fitting the model and predicting data.
Models implementations are defined in ```linear_regression_models.py```

In [4]:
from src.linear_regression.models import LinearRegressionClosedForm, LinearRegressionGradientDescent
from sklearn.linear_model import LinearRegression

sklearn_LR = LinearRegression()
closed_form_LR = LinearRegressionClosedForm()
gradient_descent_LR = LinearRegressionGradientDescent()

sklearn_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', sklearn_LR)
])

closed_form_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', closed_form_LR)
])

gradient_descent_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', gradient_descent_LR)
])

sklearn_pipeline.fit(X_train, Y_train)
closed_form_pipeline.fit(X_train, Y_train)
gradient_descent_pipeline.fit(X_train, Y_train)

y_skl = sklearn_pipeline.predict(X_test)
y_cf = closed_form_pipeline.predict(X_test)
y_gd = gradient_descent_pipeline.predict(X_test)

#### Evaluating and comparing the models.

In [5]:
from src.linear_regression.model_evaluation import metrics_table, weights_table

df_weights = weights_table(data, X, [sklearn_LR, closed_form_LR, gradient_descent_LR], ['Sklearn', 'Closed Form', 'Gradient Descent'])

df_metrics = metrics_table(Y_test, [y_skl, y_cf, y_gd], ['Sklearn', 'Closed Form', 'Gradient Descent'])

print("Comparing coefficients:")
display(df_weights)

print("Evaluating models:")
display(df_metrics)

Comparing coefficients:


,Feature,Sklearn,Closed Form,Gradient Descent
0,Hours_Studied,1.760129,1.760129,1.755704
1,Attendance,2.295587,2.295587,2.282852
2,Sleep_Hours,-0.009375,-0.009375,0.002463
3,Previous_Scores,0.716646,0.716646,0.700271
4,Tutoring_Sessions,0.590330,0.590330,0.597428
5,Physical_Activity,0.202473,0.202473,0.205796
6,Parental_Involvement - Low,1.017088,4.588674,4.590355
7,Parental_Involvement - Medium,-1.009801,2.561786,2.563665
8,Parental_Involvement - High,-0.007286,3.564300,3.557247
9,Access_to_Resources - High,0.972807,4.544394,4.547624


Evaluating models:


,R²,MSE,RMSE,MAE
Sklearn,0.693713,4.874284,2.207778,0.497735
Closed Form,0.693713,4.874284,2.207778,0.497735
Gradient Descent,0.693893,4.871424,2.207130,0.487742
